In [1]:
!pip install langchain_openai langchain_community langchain_chroma pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 51.9 MB/s eta 0:00:

In [2]:
import os
import urllib.request
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

In [3]:
os.environ['OPENAI_API_KEY'] = '여러분의 키 값'

In [4]:
# 분석할 PDF 파일을 웹에서 다운로드.
urllib.request.urlretrieve("https://github.com/llama-index-tutorial/llama-index-tutorial/raw/main/ch07/2023_%EB%B6%81%ED%95%9C%EC%9D%B8%EA%B6%8C%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf", filename="2023_북한인권보고서.pdf")

('2023_북한인권보고서.pdf', <http.client.HTTPMessage at 0x7a8f964b5b90>)

In [5]:
# LangChain의 LLM과 임베딩 모델 설정
llm = ChatOpenAI(model="gpt-4o", temperature=0.2)  # GPT-4o를 언어 모델로 사용
embed_model = OpenAIEmbeddings(model="text-embedding-3-large")  # 임베딩 모델 사용

# 문서 분할 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,  # 문서를 300자 단위로 분할
    chunk_overlap=100,  # 문맥 유지를 위해 청크 간 100자 중복
)

# PDF 문서를 읽고 벡터 인덱스 생성
loader = PyPDFLoader("2023_북한인권보고서.pdf")  # PDF 문서 로더
documents = loader.load()  # 문서에서 텍스트 추출
chunks = text_splitter.split_documents(documents)  # 문서 분할
vector_store = Chroma.from_documents(chunks, embed_model)  # 추출된 텍스트로 벡터 인덱스 생성

In [6]:
def generate_hypothetical_doc(question: str) -> str:
    """질문에 대한 가상의 이상적인 답변 문서 생성"""
    prompt = f"""주어진 질문에 대해, 마치 실제 문서에서 발췌한 것 같은 이상적인 답변을 작성해주세요.
    단, 구체적인 수치, 날짜, 트렌드와 같은 상세 정보를 포함해야 합니다.

    질문: {question}

    답변:"""

    response = llm.invoke(prompt)
    return response.content

In [7]:
def search_with_hyde(hypothetical_doc: str):
    """가상 문서를 이용해 실제 문서 검색"""
    results = vector_store.similarity_search_with_score(hypothetical_doc, k=4)
    return [
        {
            'content': doc.page_content,
            'score': score
        } for doc, score in results
    ]

In [8]:
def generate_final_answer(question: str, relevant_docs: list) -> str:
    """검색된 문서를 바탕으로 최종 답변 생성"""
    context = "\n\n".join([doc['content'] for doc in relevant_docs])

    prompt = f"""다음 검색 결과를 바탕으로 질문에 답변해주세요.
    검색 결과의 정보를 최대한 사용하고, 없는 정보는 답변하지 마세요.

    검색 결과:
    {context}

    질문: {question}

    답변:"""

    response = llm.invoke(prompt)
    return response.content

In [9]:
def process_query(question: str):
    """전체 HyDE 프로세스"""
    # 1. 가상 문서 생성
    print("1. 가상 문서 생성 중...")
    hypothetical_doc = generate_hypothetical_doc(question)
    print("\n가상 문서:", hypothetical_doc)

    # 2. 가상 문서로 검색
    print("\n2. 실제 문서 검색 중...")
    relevant_docs = search_with_hyde(hypothetical_doc)

    # 3. 최종 답변 생성
    print("\n3. 최종 답변 생성 중...")
    final_answer = generate_final_answer(question, relevant_docs)

    return {
        "hypothetical_doc": hypothetical_doc,
        "retrieved_docs": relevant_docs,
        "final_answer": final_answer
    }

In [10]:
question = "북한에서 강제로 이루어지는 조직 생활은 무엇이 있나요?"
result = process_query(question)

print("\n=== 프로세스 결과 ===")
print("\n[가상 문서]")
print(result["hypothetical_doc"])
print("\n[검색된 문서들]")
for idx, doc in enumerate(result["retrieved_docs"], 1):
    print(f"\n문서 {idx} (유사도 점수: {doc['score']:.4f}):")
    print(doc['content'])
print("\n[최종 답변]")
print(result["final_answer"])

1. 가상 문서 생성 중...

가상 문서: 북한에서 강제로 이루어지는 조직 생활은 여러 가지 형태로 존재하며, 이는 주민들의 일상생활에 깊숙이 침투해 있습니다. 대표적인 예로는 '인민반'과 '청년동맹'을 들 수 있습니다.

'인민반'은 북한 주민들이 속한 가장 기초적인 사회 조직으로, 대개 20~30가구로 구성됩니다. 인민반장은 정부의 지시를 전달하고 주민들의 동향을 감시하는 역할을 합니다. 주민들은 정기적으로 인민반 회의에 참석해야 하며, 회의에서는 정치 교육, 사회주의 이념 학습, 그리고 지역 사회의 문제 해결을 위한 논의가 이루어집니다.

또한, '사회주의애국청년동맹'은 북한의 청년들이 의무적으로 가입해야 하는 조직으로, 이들은 정기적인 모임과 행사에 참여해야 합니다. 청년동맹은 청년들에게 사회주의 이념을 주입하고, 노동력 동원 및 각종 정치적 행사에 참여하도록 강요합니다. 이러한 활동은 주로 주말이나 공휴일에 이루어지며, 참여하지 않을 경우 불이익을 받을 수 있습니다.

이 외에도, 북한 주민들은 직장 단위로 조직된 '당 세포'나 '직장 반'에 소속되어 있으며, 이들 조직을 통해 정치적 충성심을 검증받고, 사회주의 건설에 기여할 것을 요구받습니다. 이러한 조직 생활은 북한 주민들에게 개인의 자유보다는 집단의 이익을 우선시하도록 강요하며, 체제 유지의 중요한 수단으로 작용하고 있습니다.

2. 실제 문서 검색 중...

3. 최종 답변 생성 중...

=== 프로세스 결과 ===

[가상 문서]
북한에서 강제로 이루어지는 조직 생활은 여러 가지 형태로 존재하며, 이는 주민들의 일상생활에 깊숙이 침투해 있습니다. 대표적인 예로는 '인민반'과 '청년동맹'을 들 수 있습니다.

'인민반'은 북한 주민들이 속한 가장 기초적인 사회 조직으로, 대개 20~30가구로 구성됩니다. 인민반장은 정부의 지시를 전달하고 주민들의 동향을 감시하는 역할을 합니다. 주민들은 정기적으로 인민반 회의에 참석해야 하며, 회의에서는 정치 교육, 사회주의 이념 학습, 그리고 지역 사회의